# MicroController Driver Demo
This is the microcontroller driver to make a bridge between the µc and Python.  

----

In [1]:
from aad import mcd

help(mcd.MCDriver)

Help on class MCDriver in module aad.mcd:

class MCDriver(builtins.object)
 |  MCDriver(*args, **kwargs)
 |  
 |  µc driver for the Awesome Array Python Driver.
 |  
 |  
 |  ...
 |  Attributes
 |  -----------
 |  ser : serial.Serial
 |          serial port associated with the µc
 |  
 |  uc_ack_mode : ACK
 |          stores the actual ack_mode of the µc
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |      Closes the serial port if still open.
 |  
 |  __init__(self, pid=22336)
 |      Creates the driver.
 |      
 |      Details:
 |              It will search for the µc using the PID value 'DEFAULT_PID' or the one provided in argument.
 |              Takes the first found if many have the same PID.
 |              RAISE if not found.
 |      
 |      Arguments:
 |              pid: optional, the pid to search for.
 |  
 |  call_command(self, command, *args)
 |      Send a command and waits for a return value if needed
 |      
 |      Parameters:
 |              command: Th

In [2]:
print("---- List of available commands ----")
for com in mcd.CMD_LIST:
    print("*", com)

---- List of available commands ----
* CMD.SET_SR
* CMD.SET_CS
* CMD.SET_ADR_R
* CMD.SET_ADR_C
* CMD.GET_CTL
* CMD.CLK
* CMD.CLK_SR
* CMD.CLK_XNOR
* CMD.ACK_MODE
* CMD.DEBUG_ECHO
* CMD.DEBUG_LED


## Description of the commands
* `SET_SR`: ['action' command]  
    Set Shift Register input bit  
    - Argument 0: Shift Register identifier (see SR_LIST)  
    - Argument 1: Input bit state (State.RESET or State.SET)  
    
    
* `SET_CS`: ['action' command]  
    Set a Control Signal  
    - Argument 0: Control Signal identifier (see CS_LIST)  
    - Argument 1: Input bit state (State.RESET or State.SET)  
    

* `SET_ADR_R`: ['action' command]  
    Set Row address  
    - Argument 0: The 5-bit address (int or bytes)  
    
    
* `SET_ADR_R`:  ['action' command]  
    Set Column address  
    - Argument 0: The 5-bit address (int or bytes)  
    
    
* `SET_SR`:  ['action' command]  
    Set Shift Register input bit  
    - Argument 0: Shift Register identifier (see SR_LIST)  
    - Argument 1: Input bit state (State.RESET or State.SET)  
    
     
* `GET_CTL`:  ['get' command]  
    Get Shift Registers sanity bit  
    - Argument 0: Shift Register identifier (see SR_LIST)  
    - Returns the bit state (init a State with it for easier manipulation)  
    
    
* `CLK`:  ['action' command]  
    Clock once  
    
    
* `CLK_SR`:  ['action' command]  
    Clock once the shift registers  
    
    
* `CLK_XNOR`:  ['action' command]  
    Clock once the XNOR  
    

* `ACK_MODE`:  ['action' command]  
    Specify the commands which should send acknowledge after completion.  
    - Argument 0: Flag of the commands (`ACK.NONE`, `ACK.SET_SR`, `ACK.CLK`, `ACK.ALL` or any combination of (not many for now))  
    - Returns ack for `ACK_MODE`, if Argument 0 != `ACK.NONE`  
    - **Details** The ack sent is the two bytes [`0xAA`, `{CMD_CODE}`]  
    - **Details** There is no ack for commands that already returns data.
    
    
* `DEBUG_ECHO`:  ['get' command]  
    Sends back the provided argument  
    - Argument 0: byte to echo  
    - Returns the byte provided  
    
    
* `DEBUG_LED`:  ['get' command]  
    Turn on the three board LEDs for 2s  
    * Returns b'Hello, C2N!'  

----


In [3]:
print("---- List of available ports ----")
mcd.MCDriver.print_ports()

---- List of available ports ----
COM3 - Intel(R) Active Management Technology - SOL (COM3) | PID:  None
COM4 - USB Serial Device (COM4) | PID:  22336


We see that the µc is at the port `COM4`.

In [4]:
driver = mcd.MCDriver()
print("Port selected by the MCDriver:", driver.ser.port)

Port selected by the MCDriver: COM4


The MCDriver selected it by default.  

---

In [5]:
driver.debug_led()

b'Hello, C2N!'

In [6]:
driver.debug_echo(b'!')

b'!'

In [7]:
# Check WLE sanity
driver.ack_mode(mcd.ACK.CLK_SR)

driver.set_sr(mcd.SR.WLE, mcd.State.SET)
driver.clk_sr()
driver.set_sr(mcd.SR.WLE, mcd.State.RESET)

for _ in range(63):
	driver.clk_sr()

ctl = driver.get_ctl(mcd.SR.WLE)
print("This should give 'True':", ctl == mcd.State.SET)

# (Random result for now, it will not when the AwesomeArray will tested and working)

This should give 'True': False
